In [4]:
# ======================================================
# Weak-labeling para fraude financiero
# ======================================================

import os
import re
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

# ------------------------------------------------------
# 0) Subir archivo CSV
# ------------------------------------------------------
print("Sube el dataset original (CSV)")
uploaded = files.upload()
input_path = list(uploaded.keys())[0]

# ------------------------------------------------------
# 1) Leer dataset
# ------------------------------------------------------
df = pd.read_csv(input_path)

TEXT_COL = "Mensaje"
SENDER_COL = "Sender_ID"

if TEXT_COL not in df.columns:
    raise ValueError(f"No existe la columna '{TEXT_COL}'")

# ------------------------------------------------------
# 2) Normalización
# ------------------------------------------------------
def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).strip().lower()

df["text_norm"] = df[TEXT_COL].apply(normalize_text)

# ------------------------------------------------------
# 3) Diccionarios y patrones
# ------------------------------------------------------
FIN_KEYWORDS = [
    "ganancia","ganancias","gane","gané","ganar","ingreso","ingresos",
    "rentabilidad","retorno","rendimiento","capital","inversion","inversión",
    "inverti","invertí","invertir","deposit","deposito","depósito",
    "retiro","retirar","pago","pagos","transfer","transferencia",
    "profit","trading","broker","crypto","bitcoin","usdt"
]

CERTAINTY_KEYWORDS = [
    "garantizado","seguro","sin riesgo","100%","asegurado","confiable"
]

FIRST_PERSON = ["yo","me","mi","conmigo","empecé","empece","inicié","inicie"]
ACHIEVEMENT_VERBS = ["gané","gane","logré","logre","conseguí","consegui","recibí","recibi","generé","genere"]

BEFORE_AFTER_PATTERNS = [
    r"al principio.*ahora",
    r"antes.*ahora",
    r"al inicio.*ahora"
]

PRIVATE_CONTACT_PATTERNS = [
    r"\bescrib(e|í|eme)\b|\bescribeme\b|\bescríbeme\b",
    r"\bcontact(a|á)me\b|\bcontactame\b|\bcontáctame\b",
    r"\b(dm|inbox|privado|al privado|mensaje privado|md)\b",
    r"\bmas info\b|\bmás info\b|\binformaci(o|ó)n\b|\bte explico\b"
]

MONEY_EMOJIS = {"💰","🤑","💵","💸","📈","🚀","🎉","🏦","💳","✅"}
HYPE_WORDS = [
    "increible","increíble","impresionante","excelente",
    "fantástico","fantastico","oportunidad","rápido",
    "facil","fácil","urgente","no te arrepentirás"
]

# ------------------------------------------------------
# 4) Funciones auxiliares
# ------------------------------------------------------
def count_keywords(text, keywords):
    return sum(1 for k in keywords if k in text)

def regex_match(text, patterns):
    return any(re.search(p, text, re.IGNORECASE) for p in patterns)

def count_emojis(text):
    return sum(1 for c in str(text) if c in MONEY_EMOJIS)

# ------------------------------------------------------
# 5) Reglas R1–R4
# ------------------------------------------------------
def R1(text):
    k = count_keywords(text, FIN_KEYWORDS)
    pts = 3 if k >= 2 else 2 if k == 1 else 0
    if pts > 0 and count_keywords(text, CERTAINTY_KEYWORDS) > 0:
        pts += 1
    return pts

def R2(text):
    pts = 0
    if any(p in text for p in FIRST_PERSON) and any(v in text for v in ACHIEVEMENT_VERBS):
        pts += 1
    if regex_match(text, BEFORE_AFTER_PATTERNS):
        pts += 2
    return pts

def R3(text):
    return 3 if regex_match(text, PRIVATE_CONTACT_PATTERNS) else 0

def R4(raw, norm):
    pts = 0
    if count_emojis(raw) >= 2:
        pts += 2
    if any(h in norm for h in HYPE_WORDS) and count_emojis(raw) >= 1:
        pts += 1
    return min(3, pts)

df["R1"] = df["text_norm"].apply(R1)
df["R2"] = df["text_norm"].apply(R2)
df["R3"] = df["text_norm"].apply(R3)
df["R4"] = [R4(r, n) for r, n in zip(df[TEXT_COL], df["text_norm"])]

# ------------------------------------------------------
# 6) Regla R5 – repetición por usuario
# ------------------------------------------------------
df["R5"] = 0

if SENDER_COL in df.columns:
    vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(3,5), min_df=2)
    X = vectorizer.fit_transform(df["text_norm"])

    for sender, idxs in df.groupby(SENDER_COL).indices.items():
        if len(idxs) > 1:
            sims = cosine_similarity(X[idxs])
            np.fill_diagonal(sims, 0)
            df.loc[idxs, "R5"] = (sims.max(axis=1) >= 0.85).astype(int) * 3

# ------------------------------------------------------
# 7) Score y etiqueta final
# ------------------------------------------------------
df["fraud_score"] = df[["R1","R2","R3","R4","R5"]].sum(axis=1)
df["fraud_label"] = (df["fraud_score"] >= 5).astype(int)

# ------------------------------------------------------
# 8) Guardar y DESCARGAR automáticamente
# ------------------------------------------------------
output_file = "dataset_etiquetado.csv"
df.drop(columns=["text_norm"]).to_csv(output_file, index=False, encoding="utf-8-sig")

print("Descargando dataset_etiquetado.csv ...")
files.download(output_file)

print("\nDistribución final de etiquetas:")
print(df["fraud_label"].value_counts(normalize=True))


Sube el dataset original (CSV)


Saving datasetFraude.csv to datasetFraude (2).csv
Descargando dataset_etiquetado.csv ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Distribución final de etiquetas:
fraud_label
0    0.80761
1    0.19239
Name: proportion, dtype: float64


# ======================================================
# RE-ETIQUETADO (weak supervision) con reglas R1–R9
# Incluye nuevas reglas: R6, R7, R8, R9
# - Sube un CSV en Google Colab
# - Genera: R1..R9, fraud_score, fraud_label
# - Guarda como: dataset_etiquetado_v2.csv
# - DESCARGA automáticamente al finalizar
#
# Requisitos esperados en el CSV:
#   - Columna de texto: "Mensaje"
#   - (Opcional) Columna usuario: "Sender_ID" (mejora R5 y R9)
#
# Ajusta THRESHOLD si deseas más/menos positivos
# ======================================================

In [1]:
# ======================================================
# RE-ETIQUETADO (weak supervision) con reglas R1–R9
# Incluye nuevas reglas: R6, R7, R8, R9
# - Sube un CSV en Google Colab
# - Genera: R1..R9, fraud_score, fraud_label
# - Guarda como: dataset_etiquetado_v2.csv
# - DESCARGA automáticamente al finalizar
#
# Requisitos esperados en el CSV:
#   - Columna de texto: "Mensaje"
#   - (Opcional) Columna usuario: "Sender_ID" (mejora R5 y R9)
#
# Ajusta THRESHOLD si deseas más/menos positivos
# ======================================================

import re
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

from google.colab import files

# -----------------------------
# 0) Subir archivo
# -----------------------------
print("Sube el dataset CSV (original o dataset_etiquetado.csv).")
uploaded = files.upload()
input_path = list(uploaded.keys())[0]

df = pd.read_csv(input_path)

TEXT_COL = "Mensaje"
SENDER_COL = "Sender_ID"  # opcional, si no existe se degradan R5 y R9

if TEXT_COL not in df.columns:
    raise ValueError(f"No existe la columna '{TEXT_COL}'. Columnas: {list(df.columns)}")

df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str)

# -----------------------------
# 1) Normalización (para matching de reglas)
# -----------------------------
def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).strip().lower()

df["text_norm"] = df[TEXT_COL].apply(normalize_text)

# -----------------------------
# 2) Diccionarios / patrones
# -----------------------------
# R1: financiero explícito
FIN_KEYWORDS = [
    "ganancia","ganancias","gane","gané","ganar","ingreso","ingresos",
    "rentabilidad","retorno","rendimiento","capital","inversion","inversión",
    "inverti","invertí","invertir","deposit","deposito","depósito",
    "retiro","retirar","pago","pagos","transfer","transferencia",
    "profit","trading","broker","crypto","bitcoin","usdt","cripto","criptomoneda"
]
CERTAINTY_KEYWORDS = ["garantizado","seguro","sin riesgo","100%","asegurado","confiable"]

# R2: testimonial
FIRST_PERSON = ["yo","me","mi","conmigo","mío","empecé","empece","inicié","inicie"]
ACHIEVEMENT_VERBS = ["gané","gane","logré","logre","conseguí","consegui","recibí","recibi","generé","genere"]
BEFORE_AFTER_PATTERNS = [r"al principio.*ahora", r"antes.*ahora", r"al inicio.*ahora"]

# R3/R7: contacto / llamado a acción (ampliado)
PRIVATE_CONTACT_PATTERNS = [
    r"\b(escr(i|í)b(e|eme)|escribeme|escríbeme)\b",
    r"\b(cont(a|á)ct(a|á)me|contactame|contáctame)\b",
    r"\b(dm|d\.m\.|inbox|privado|al privado|por privado|mensaje privado|md)\b",
    r"\b(whatsapp|wsp|wa\.me)\b",
    r"\b(m(a|á)s info|info|informaci(o|ó)n|detalles)\b",
    r"\b(te explico|te indico|te cuento|te asesoro)\b",
    r"\b(interesad[oa]s?|cupos|cupo limitado|aplica)\b"
]

# R4: emojis/hype
MONEY_EMOJIS = {"💰","🤑","💵","💸","📈","🚀","🎉","🏦","💳","✅"}
HYPE_WORDS = [
    "increible","increíble","impresionante","excelente","fantastico","fantástico",
    "oportunidad","aprovecha","rápido","facil","fácil","urgente","no te arrepentirás","no te arrepentiras",
    "ganancias aseguradas","ingresos extra"
]

# R6: “fraude implícito” (captación)
IMPLICIT_FINANCE = [
    "oportunidad","proyecto","negocio","plataforma","programa","sistema","esquema",
    "trabajo online","trabajo en linea","online","en línea","ingresos extra","extra",
    "emprendimiento","afiliados","referidos","comisión","comisiones","network","networking"
]

# R8: mensajes cortos genéricos (captación) + contacto
MAX_SHORT_LEN = 120

# -----------------------------
# 3) Helpers
# -----------------------------
def count_keywords(text, keywords):
    return sum(1 for k in keywords if k in text)

def regex_any(text, patterns):
    return any(re.search(p, text, flags=re.IGNORECASE | re.UNICODE) for p in patterns)

def count_money_emojis(raw_text):
    if pd.isna(raw_text):
        return 0
    s = str(raw_text)
    return sum(1 for ch in s if ch in MONEY_EMOJIS)

# -----------------------------
# 4) Reglas R1–R8 por mensaje
# -----------------------------
def rule_R1_financial(text_norm):
    k_fin = count_keywords(text_norm, FIN_KEYWORDS)
    k_cert = count_keywords(text_norm, CERTAINTY_KEYWORDS)

    if k_fin >= 2:
        pts = 3
    elif k_fin == 1:
        pts = 2
    else:
        pts = 0

    if pts > 0 and k_cert >= 1:
        pts += 1  # boost moderado

    return pts

def rule_R2_testimonial(text_norm):
    pts = 0
    has_fp = any(fp in text_norm for fp in FIRST_PERSON)
    has_ach = any(v in text_norm for v in ACHIEVEMENT_VERBS)
    if has_fp and has_ach:
        pts += 1
    if regex_any(text_norm, BEFORE_AFTER_PATTERNS):
        pts += 2
    return pts

def rule_R3_private_contact(text_norm):
    return 3 if regex_any(text_norm, PRIVATE_CONTACT_PATTERNS) else 0

def rule_R4_emotion_emojis(raw_text, text_norm):
    pts = 0
    n_emj = count_money_emojis(raw_text)

    if n_emj >= 2:
        pts += 2

    has_hype = any(hw in text_norm for hw in HYPE_WORDS)
    if n_emj >= 1 and has_hype:
        pts += 1

    # cap para que no domine
    return min(3, pts)

# NUEVA R6: captación implícita (sin decir “gané”)
def rule_R6_implicit_finance(text_norm):
    k_impl = count_keywords(text_norm, IMPLICIT_FINANCE)
    # +1 si hay al menos una señal, +2 si hay varias (más fuerte)
    if k_impl >= 2:
        return 2
    elif k_impl == 1:
        return 1
    return 0

# NUEVA R7: llamado a acción sin contexto financiero explícito
def rule_R7_action_without_finance(text_norm):
    has_action = regex_any(text_norm, PRIVATE_CONTACT_PATTERNS)
    k_fin = count_keywords(text_norm, FIN_KEYWORDS)
    k_impl = count_keywords(text_norm, IMPLICIT_FINANCE)
    # si llama a contacto pero NO hay contexto financiero (ni explícito ni implícito) -> sospechoso
    if has_action and (k_fin == 0) and (k_impl == 0):
        return 2
    return 0

# NUEVA R8: mensaje corto genérico + hype/captación + contacto
def rule_R8_short_generic(raw_text, text_norm):
    if len(str(raw_text)) > MAX_SHORT_LEN:
        return 0
    has_action = regex_any(text_norm, PRIVATE_CONTACT_PATTERNS)
    has_hype = any(hw in text_norm for hw in HYPE_WORDS)
    has_impl = count_keywords(text_norm, IMPLICIT_FINANCE) >= 1
    if has_action and (has_hype or has_impl):
        return 2
    return 0

df["R1"] = df["text_norm"].apply(rule_R1_financial)
df["R2"] = df["text_norm"].apply(rule_R2_testimonial)
df["R3"] = df["text_norm"].apply(rule_R3_private_contact)
df["R4"] = [rule_R4_emotion_emojis(raw, norm) for raw, norm in zip(df[TEXT_COL], df["text_norm"])]
df["R6"] = df["text_norm"].apply(rule_R6_implicit_finance)
df["R7"] = df["text_norm"].apply(rule_R7_action_without_finance)
df["R8"] = [rule_R8_short_generic(raw, norm) for raw, norm in zip(df[TEXT_COL], df["text_norm"])]

# -----------------------------
# 5) R5: repetición por usuario (si existe Sender_ID)
# -----------------------------
df["R5"] = 0
if SENDER_COL in df.columns:
    # char n-grams robusto a pequeñas variaciones
    vect_sender = TfidfVectorizer(analyzer="char_wb", ngram_range=(3, 5), min_df=2)
    X_sender = vect_sender.fit_transform(df["text_norm"])

    sim_threshold_sender = 0.85
    r5 = np.zeros(len(df), dtype=int)

    for sender, idxs in df.groupby(SENDER_COL).indices.items():
        if len(idxs) < 2:
            continue
        Xi = X_sender[idxs]
        S = cosine_similarity(Xi)
        np.fill_diagonal(S, 0.0)
        max_sim = S.max(axis=1)
        r5_sender = (max_sim >= sim_threshold_sender).astype(int) * 3
        r5[idxs] = r5_sender

    df["R5"] = r5

# -----------------------------
# 6) NUEVA R9: repetición/campaña entre usuarios distintos
#     (NearestNeighbors para evitar O(n^2))
# -----------------------------
df["R9"] = 0
if SENDER_COL in df.columns:
    # Representación para "campañas": char-ngrams funciona muy bien en spam coordinado
    vect_campaign = TfidfVectorizer(analyzer="char_wb", ngram_range=(3, 5), min_df=2)
    Xc = vect_campaign.fit_transform(df["text_norm"])

    # Vecinos cercanos por coseno (distancia coseno; similitud = 1 - distancia)
    # k pequeño para rendimiento
    k = 6  # incluye self + 5 vecinos
    nn = NearestNeighbors(n_neighbors=min(k, Xc.shape[0]), metric="cosine", algorithm="brute")
    nn.fit(Xc)

    distances, indices = nn.kneighbors(Xc, return_distance=True)

    # Umbral de campaña entre usuarios distintos
    campaign_sim_threshold = 0.90

    senders = df[SENDER_COL].values

    r9 = np.zeros(len(df), dtype=int)
    for i in range(len(df)):
        si = senders[i]
        # saltar el vecino 0 (normalmente es el mismo punto)
        flagged = False
        for dist, j in zip(distances[i][1:], indices[i][1:]):
            sim = 1.0 - float(dist)
            if sim >= campaign_sim_threshold and senders[j] != si:
                flagged = True
                break
        r9[i] = 3 if flagged else 0

    df["R9"] = r9

# -----------------------------
# 7) Score total y etiqueta final
# -----------------------------
RULE_COLS = ["R1","R2","R3","R4","R5","R6","R7","R8","R9"]
df["fraud_score"] = df[RULE_COLS].sum(axis=1)

# IMPORTANTE: umbral más sensible (para subir recall)
# Si sospechas que hay muy pocos positivos, usa 3 (recomendado).
THRESHOLD = 3
df["fraud_label"] = (df["fraud_score"] >= THRESHOLD).astype(int)

# -----------------------------
# 8) Reporte rápido (para confirmar tu sospecha)
# -----------------------------
print("\n=== Distribución fraud_label (v2) ===")
counts = df["fraud_label"].value_counts().sort_index()
print(counts)
print("\nProporción fraude:", round(df["fraud_label"].mean(), 4))

print("\n=== Promedios por regla ===")
print(df[RULE_COLS].mean().sort_values(ascending=False))

print("\n=== Top 10 por fraud_score (para sanity check) ===")
show_cols = [c for c in ["Fecha", SENDER_COL, TEXT_COL] if c in df.columns] + ["fraud_score","fraud_label"] + RULE_COLS
print(df.sort_values("fraud_score", ascending=False).head(10)[show_cols].to_string(index=False))

# -----------------------------
# 9) Guardar y descargar
# -----------------------------
output_file = "dataset_etiquetado_v2.csv"
df.drop(columns=["text_norm"]).to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"\nArchivo generado: {output_file}")
print("Descargando automáticamente...")

files.download(output_file)


Sube el dataset CSV (original o dataset_etiquetado.csv).


Saving datasetFraude.csv to datasetFraude (3).csv

=== Distribución fraud_label (v2) ===
fraud_label
0    1490
1    1795
Name: count, dtype: int64

Proporción fraude: 0.5464

=== Promedios por regla ===
R1    1.630441
R5    0.595434
R3    0.284932
R6    0.249619
R2    0.106849
R9    0.100457
R4    0.099543
R7    0.090715
R8    0.008524
dtype: float64

=== Top 10 por fraud_score (para sanity check) ===
                    Fecha  Sender_ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>